## FSL automated script

----
Pablo Lasito

---

check [Andy's video](https://www.andysbrainblog.com/andysbrainblog/tag/DTI)

In [35]:
## author: Laso
## date: Feb, 18th

import os

# check your home directory with this:
from pathlib import Path
print(Path.home())
# set main as the path to nifty directory:
sep=os.path.sep
main= sep+'Volumes'+sep+'HDD1'+sep+'INVESTIGACION'+sep+'SENECA-PICASSO'+sep+'nifti'

import json
def json_params(files,save_dir,name,ckpt=0):
    texts=[]
    for file in files:
        with open(file) as f:
            d = json.load(f)
            AT=(d['AcquisitionMatrixPE'])
            ES=d['EffectiveEchoSpacing']
            j_direction=d['PhaseEncodingDirection']
            ReadOut_Time =AT*ES*0.5
            text= j_direction.replace('j-','0 -1 0 ').replace('j','0 1 0 ')+ str(ReadOut_Time)
            texts.append(text)
        with open(os.path.join(save_dir, name), 'w') as f:
            for item in texts:
                f.write("%s\n" % item)
    if ckpt:
        print('file saved as: ', save_dir+sep+name)
        print('content saved: ', texts)
        
import subprocess
def run_cmd(cmd,disp=0):
    lasito=subprocess.Popen(cmd,stdout=subprocess.PIPE,shell=True)
    out, err = lasito.communicate()
    if disp:#ckpt
        print('>',cmd)
        print('>>',out,err)
    return out,err

/Users/administrator


In [42]:
mario = input('proceed with '+ sb+ses+'? \n --> If so, enter "yes". \n --> If not, enter "no". \n')
if mario=='yes':
    print('hooola')

proceed with sub-sp02ses-02? 
 --> If so, enter "yes". 
 --> If not, enter "no". 
yes
hooola


---

In [23]:
import fnmatch
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces import fsl
from nipype.testing import example_data
from nipype.interfaces import fsl
from nipype.testing import example_data
from nipype.interfaces.fsl import Merge
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP

Subjs=[]
for subj in os.listdir(main):
    if subj.startswith('sub'):
        #print(subj)#ckpt
        Subjs.append(subj)
   
for sb in (Subjs):
    for ses in os.listdir(main+sep+sb):
        if 'ses' in ses:
            print('>> ',sb,ses,':')
            
            directory_dwi=main+sep+sb+sep+ses+sep+'dwi'
            directory_func=main+sep+sb+sep+ses+sep+'func'
            files=[directory_dwi+sep+sb+ses+'_dir-AP_dwi.json',directory_func+sep+sb+ses+'_task-rest_bold.json']
            json_params(files,main,'acq_params-'+sb+ses+'.txt')
            print('params created')
               
            # get one volume for dwi.nii
            dwi_file=main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.nii'
            output_file=main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'
            fslroi = ExtractROI(in_file=dwi_file, roi_file=output_file, t_min=0,t_size=1)
            fslroi.cmdline #== 'fslroi %s nodif.nii 0 1' % dwidata
            fslroi.run()
            print('>> fslroi_dwi DONE')
            
            bold_file=main+sep+sb+sep+ses+sep+'func'+sep+sb+ses+'_task-rest_bold.nii'
            output_file=main+sep+sb+sep+ses+sep+'func'+sep+'nodif_'+sb+ses+'-bold.nii.gz'
            fslroi = ExtractROI(in_file=bold_file, roi_file=output_file, t_min=0,t_size=1)
            fslroi.cmdline #== 'fslroi %s nodif.nii 0 1' % dwidata
            fslroi.run()
            print('> fslroi_func DONE')
            

            dwi_file =main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'
            bold_file=main+sep+sb+sep+ses+sep+'func'+sep+'nodif_'+sb+ses+'-bold.nii.gz'
            flt = fsl.FLIRT(bins=640, cost_func='mutualinfo')
            flt.inputs.in_file = dwi_file
            flt.inputs.reference = bold_file
            flt.inputs.output_type = "NIFTI_GZ"
            flt.cmdline
            #'flirt -in structural.nii -ref mni.nii -out structural_flirt.nii.gz -omat structural_flirt.mat -bins 640 -searchcost mutualinfo'
            res = flt.run()
            print('> flirt DONE (not that flirting, Mario...)')
            # MOVE FLIES!!!!!
            
                        
            merger = Merge()
            dwi_file= main+sep+'nodif_'+sb+ses+'-dwi_flirt.nii.gz'
            bold_file=main+sep+sb+sep+ses+sep+'func'+sep+'nodif_'+sb+ses+'-bold.nii.gz'
            merger.inputs.in_files = [dwi_file, bold_file]
            merger.inputs.dimension = 't'
            merger.inputs.output_type = 'NIFTI_GZ'
            merger.cmdline
            #'fslmerge -t functional2_merged.nii.gz functional2.nii functional3.nii'
            res=merger.run()
            print('> merge DONE')
            # MOVE FLIES!!!!!
            
            
            imain=main+sep+'nodif_'+sb+ses+'-dwi_flirt_merged.nii.gz'
            params=main+sep+'acq_params-'+sb+ses+'.txt'
            topup = TOPUP()
            topup.inputs.in_file = imain
            topup.inputs.encoding_file = params
            topup.inputs.output_type = "NIFTI_GZ"
            topup.cmdline 
            #'topup --config=b02b0.cnf --datain=topup_encoding.txt --imain=b0_b0rev.nii --out=b0_b0rev_base --iout=b0_b0rev_corrected.nii.gz --fout=b0_b0rev_field.nii.gz --logout=b0_b0rev_topup.log'
            res = topup.run()
            print('> TOPUP completed. Please, check your fieldmap.')
            
            applytopup = ApplyTOPUP()
            #nodif_sub-sp01ses-02-dwi_flirt_merged_base_fieldcoef.nii.gz
            imain_up= main+sep+'nodif_'+sb+ses+'-dwi_flirt.nii.gz'
            imain_down=main+sep+sb+sep+ses+sep+'func'+sep+'nodif_'+sb+ses+'-bold.nii.gz'
            #imain_up  = main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.nii' #dwi_file
            #imain_down= main+sep+sb+sep+ses+sep+'func'+sep+sb+ses+'_task-rest_bold.nii' #bold_file
            applytopup.inputs.in_files = [imain_up,imain_down]
            applytopup.inputs.encoding_file = main+sep+'acq_params-'+sb+ses+'.txt' #params
            applytopup.inputs.in_topup_fieldcoef = main+sep+'nodif_'+sb+ses+'-dwi_flirt_merged_base_fieldcoef.nii.gz'
            applytopup.inputs.in_topup_movpar = main+sep+'nodif_'+sb+ses+'-dwi_flirt_merged_base_movpar.txt'
            applytopup.inputs.output_type = "NIFTI_GZ"
            applytopup.cmdline 
            res = applytopup.run() 
            print('> ApplyTopup completed. Please, check final outcome.')

>  sub-sp01 ses-01 :
>> fslroi_dwi DONE
>> fslroi_func DONE
>  sub-sp01 ses-02 :
>> fslroi_dwi DONE
>> fslroi_func DONE
>  sub-sp02 ses-01 :
>> fslroi_dwi DONE
>> fslroi_func DONE
>  sub-sp02 ses-02 :
>> fslroi_dwi DONE
>> fslroi_func DONE


In [34]:
from nipype.interfaces.fsl import ApplyTOPUP
Subjs=[]
for subj in os.listdir(main):
    if subj.startswith('sub'):
        #print(subj)#ckpt
        Subjs.append(subj)
# nodif_sub-sp01ses-01-dwi_flirt_merged.nii.gz   
for sb in (Subjs):
    for ses in os.listdir(main+sep+sb):
        if 'ses' in ses:
            print('>> ',sb,ses,':')
            
            applytopup = ApplyTOPUP()
            #nodif_sub-sp01ses-02-dwi_flirt_merged_base_fieldcoef.nii.gz
            imain_up= main+sep+'nodif_'+sb+ses+'-dwi_flirt.nii.gz'
            imain_down=main+sep+sb+sep+ses+sep+'func'+sep+'nodif_'+sb+ses+'-bold.nii.gz'
            #imain_up  = main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.nii' #dwi_file
            #imain_down= main+sep+sb+sep+ses+sep+'func'+sep+sb+ses+'_task-rest_bold.nii' #bold_file
            applytopup.inputs.in_files = [imain_up,imain_down]
            applytopup.inputs.encoding_file = main+sep+'acq_params-'+sb+ses+'.txt' #params
            applytopup.inputs.in_topup_fieldcoef = main+sep+'nodif_'+sb+ses+'-dwi_flirt_merged_base_fieldcoef.nii.gz'
            applytopup.inputs.in_topup_movpar = main+sep+'nodif_'+sb+ses+'-dwi_flirt_merged_base_movpar.txt'
            applytopup.inputs.output_type = "NIFTI_GZ"
            applytopup.cmdline 
            res = applytopup.run() 
            print('> ApplyTopup completed. Please, check final outcome.')



>>  sub-sp01 ses-01 :
> ApplyTopup completed. Please, check final outcome.
>>  sub-sp01 ses-02 :
> ApplyTopup completed. Please, check final outcome.
>>  sub-sp02 ses-01 :
> ApplyTopup completed. Please, check final outcome.
>>  sub-sp02 ses-02 :
> ApplyTopup completed. Please, check final outcome.


---

---

In [3]:
'''
import fnmatch
Subjs=[]
for subj in os.listdir(main):
    if 'sub' in subj:
        #print(subj)#ckpt
        Subjs.append(subj)
   
for sb in (Subjs):
    for ses in os.listdir(main+sep+sb):
        if 'ses' in ses:
            print('>> ',sb,ses,':')
'''  
            
            
            
########################################################            
# previuos code --ª DO NOT DELETE (it might be useful) #   
######################################################## 
'''
for i in range(len(Subjs)):
    
    
    gen = (x for x in xyz if x not in a)
    for ses in os.listdir(main+sep+Subjs[i]):
        print('\n>>',Subjs[i],ses,'...') #ckpt
        if 'ses' in ses:
            Ses.append(ses)
            print(ses)
        directory_fmap=main+sep+Subjs[i]+sep+ses+sep+'fmap'
        
        directory_func=main+sep+Subjs[i]+sep+ses+sep+'func'
        if os.path.exists((directory_func)): # make txt file for params (later needed topup command)
            directory_func=main+sep+Subjs[i]+sep+ses+sep+'func'
            json_params(directory_func+sep+Subjs[i]+ses+'_task-rest_bold.json',directory_func,'acq_params'+Subjs[i]+ses+'.txt')
            
        if os.path.exists(directory_func): # take direction to bold.nii
            for file_name in os.listdir(directory_func):
                if fnmatch.fnmatch(file_name, 'sub-sp*task-rest_bold.nii'):
                    print('>>>',file_name) #ckpt
                    bold_file=(directory_func+sep+file_name)                    
                    #print('>>>',file_name,'IN',dwi_file) #ckpt
                    
        directory_dwi=main+sep+Subjs[i]+sep+ses+sep+'dwi'
        if os.path.exists(directory_dwi): # take directory to dwi.nii
            for file_name in os.listdir(directory_dwi):
                if fnmatch.fnmatch(file_name, '*dwi.nii'):
                    print('>>>',file_name) #ckpt
                    dwi_file=(directory_dwi+sep+file_name)
        
        if os.path.exists(directory_func) and os.path.exists(directory_dwi):
            cmd='fslroi'+' "'+dwi_file+'" ' +'nodif '+'0 1' #get one volume
            run_cmd(cmd,1)
            cmd='flirt '+'-in' +' "'+dwi_file+'" '+' "'+bold_file+'" ' +'nodif_PA' # dwi to same dimension as bold
            run_cmd(cmd,1)
            #print('>>>>entering command: ',cmd)#ckpt
                    
            merged_output='AP_PA_b0'
            cmd='fsl merge '+ '-t ' + merged_output + ' nodif ' + 'nodif_PA' # merge into one
            run_cmd(cmd,1)
        
            cmd='topup '+ '--imain='+merged_output + ' datain='+'acq_params*.txt ' + '--config=b02b0.cnf ' +'--out='+directory_fmap+'acq_rest-fmap' 
            run_cmd(cmd,1)
                
'''   

>>  sub-sp01 ses-01 :
>>  sub-sp01 ses-02 :
>>  sub-sp02 ses-01 :
>>  sub-sp02 ses-02 :


'\nfor i in range(len(Subjs)):\n    \n    \n    gen = (x for x in xyz if x not in a)\n    for ses in os.listdir(main+sep+Subjs[i]):\n        print(\'\n>>\',Subjs[i],ses,\'...\') #ckpt\n        if \'ses\' in ses:\n            Ses.append(ses)\n            print(ses)\n        directory_fmap=main+sep+Subjs[i]+sep+ses+sep+\'fmap\'\n        \n        directory_func=main+sep+Subjs[i]+sep+ses+sep+\'func\'\n        if os.path.exists((directory_func)): # make txt file for params (later needed topup command)\n            directory_func=main+sep+Subjs[i]+sep+ses+sep+\'func\'\n            json_params(directory_func+sep+Subjs[i]+ses+\'_task-rest_bold.json\',directory_func,\'acq_params\'+Subjs[i]+ses+\'.txt\')\n            \n        if os.path.exists(directory_func): # take direction to bold.nii\n            for file_name in os.listdir(directory_func):\n                if fnmatch.fnmatch(file_name, \'sub-sp*task-rest_bold.nii\'):\n                    print(\'>>>\',file_name) #ckpt\n                  

---
Pablo Laso Mielgo